## Mikan-tutorial

Welcome to mikan 🍊!

Mikan is a **m**edical **i**mage **k**it for segment**a**tion metrics evaluatio**n**, native Rust support, and Python bindings for cross-language performance. This document introduces how to use mikan and briefly compares its performance and results with medpy.

Main features of mikan:

- Rust-driven, highly parallelized, and extremely fast, 10 to 100 times faster than existing tools (such as medpy)
- Supports the calculation of almost all segmentation metrics
- Carefully designed simple, flexible, and intuitive interface, you can master mikan and fall in love with it in just one minute!


### TL;DR

Run mikan immediately to get all metrics for all segmentation labels:

In [1]:
from mikan import all

all_metrics = all(rf"..\data\patients_26_ground_truth.nii.gz", rf"..\data\patients_26_segmentation.nii.gz")
print(all_metrics)

[{'accuracy': 0.99887297469187, 'adjust_rand_score': 0.9864214165654426, 'amcc': 0.9864367567296637, 'assd': 0.4627411239979557, 'auc': 0.9907127577891529, 'balanced_accuracy': 0.9907127577891529, 'dice': 0.9870104666571462, 'f_score': 0.9870104666571462, 'fn': 136914.0, 'fnr': 0.018227270960283256, 'fp': 57192.0, 'fpr': 0.00034721346141101695, 'hausdorff_distance': 23.23409712643984, 'hausdorff_distance_95': 1.161103780937166, 'jaccard_score': 0.9743540620800129, 'kappa': 0.9864214165654426, 'label': 1.0, 'masd': 0.4570258359112762, 'mcc': 0.9864367567296637, 'nmcc': 0.9932183783648318, 'precision': 0.9923043904749475, 'senstivity': 0.9817727290397168, 'specificity': 0.999652786538589, 'tn': 164659924.0, 'tp': 7374578.0, 'volume_similarity': 0.9946650202613058}, {'accuracy': 0.9994664649440818, 'adjust_rand_score': 0.9635612411025513, 'amcc': 0.963574881429292, 'assd': 0.6030765488980315, 'auc': 0.9843198140671667, 'balanced_accuracy': 0.9843198140671667, 'dice': 0.9638299832080431, '

### Usage

In [2]:
import mikan
import SimpleITK as sitk
import time
from rich import print

Currently, mikan only supports uint8 data, which is usually sufficient for most tasks.

In [3]:
gt = sitk.ReadImage(rf"..\data\patients_26_ground_truth.nii.gz", sitk.sitkUInt8)
pred = sitk.ReadImage(rf"..\data\patients_26_segmentation.nii.gz", sitk.sitkUInt8)

Using mikan requires just two steps:

1. Initialize an evaluator:

In [4]:
e = mikan.Evaluator(gt, pred)

2. Obtain metrics:

In [5]:
e.labels(1).metrics("dice") # one label, one metric

0.9870104666571462

In [6]:
e.labels([1,2,3]).metrics("Dice") # multi labels, one metric

[0.9870104666571462, 0.9638299832080431, 0.9788861062581474]

In [7]:
e.labels(1).metrics(["Dice", "TP"]) # one label, multi metrics

[0.9870104666571462, 7374578.0]

In [8]:
e.labels([1, 2]).metrics(["Dice", "TP"]) # multi labels, multi metrics

{'1': {'Dice': 0.9870104666571462, 'TP': 7374578.0},
 '2': {'Dice': 0.9638299832080431, 'TP': 1224306.0}}

In [9]:
e.labels("all").metrics(["Dice", "TP"]) # all labels, multi metrics

{'1': {'Dice': 0.9870104666571462, 'TP': 7374578.0},
 '2': {'Dice': 0.9638299832080431, 'TP': 1224306.0},
 '3': {'Dice': 0.9788861062581474, 'TP': 4379325.0},
 '4': {'Dice': 0.9847150871159485, 'TP': 6222520.0},
 '5': {'Dice': 0.9799666340747192, 'TP': 4541233.0}}

In [10]:
e.labels("all").metrics("all") # all labels, all metrics

{'1': {'precision': 0.9923043904749475,
  'masd': 0.4570258359112762,
  'assd': 0.4627411239979557,
  'fp': 57192.0,
  'adjust_rand_score': 0.9864214165654426,
  'jaccard_score': 0.9743540620800129,
  'auc': 0.9907127577891529,
  'f_score': 0.9870104666571462,
  'senstivity': 0.9817727290397168,
  'dice': 0.9870104666571462,
  'balanced_accuracy': 0.9907127577891529,
  'fnr': 0.018227270960283256,
  'kappa': 0.9864214165654426,
  'specificity': 0.999652786538589,
  'tp': 7374578.0,
  'mcc': 0.9864367567296637,
  'accuracy': 0.99887297469187,
  'amcc': 0.9864367567296637,
  'fpr': 0.00034721346141101695,
  'nmcc': 0.9932183783648318,
  'fn': 136914.0,
  'hausdorff_distance': 23.23409712643984,
  'hausdorff_distance_95': 1.161103780937166,
  'volume_similarity': 0.9946650202613058,
  'tn': 164659924.0},
 '2': {'precision': 0.9587661320636511,
  'masd': 0.5948439308029991,
  'assd': 0.6030765488980315,
  'fp': 52654.0,
  'adjust_rand_score': 0.9635612411025513,
  'jaccard_score': 0.930185

### Efficiency


Although calculating the Dice coefficient is straightforward, mikan is highly optimized to make this calculation exceptionally fast. 

For example:

In [11]:
from medpy.metric import dc

e = mikan.Evaluator(gt, pred)

# mikan: DSC
t = time.time()
mikan_dscs = e.labels([1,2,3,4,5]).metrics("dsc")
mikan_costs = time.time() - t

# medpy: DSC
t = time.time()

pred_arr = sitk.GetArrayFromImage(pred)
gt_arr = sitk.GetArrayFromImage(gt)

dscs = []
for i in (1,2,3,4,5):
    dscs.append(dc(pred_arr == i, gt_arr == i))
medpy_costs = time.time() - t

print(f"mikan DSC: {medpy_costs / mikan_costs :.2f}x faster")
print(f"mikan DSC: {mikan_dscs}")
print(f"medpy DSC: {dscs}")

mikan DSC: 8.92x faster

mikan DSC: [0.9870104666571462, 0.9638299832080431, 0.9788861062581474, 0.9847150871159485, 0.9799666340747192]

medpy DSC: [0.9870104666571462, 0.9638299832080431, 0.9788861062581474, 0.9847150871159485, 0.9799666340747192]

Mikan offers ultra-fast Hausdorff distance calculations:：

In [12]:
t = time.time()
mikan_hd = e.labels(1).metrics("hd")
print(f"Mikan cost {time.time() - t:.2f} s, HD = {mikan_hd:.4f} mm")

Mikan cost 4.15 s, HD = 23.2341 mm

Comparison with medpy:

In [13]:
from medpy.metric import hd
t = time.time()
medpy_hd = hd(pred_arr == 1, gt_arr == 1, voxelspacing=gt.GetSpacing()[::-1]) # z, y, x
print(f"medpy cost {time.time() - t:.4f} s, HD = {medpy_hd:.4f} mm")

medpy cost 132.0814 s, HD = 23.2341 mm

### Super Caching

When calculating the Hausdorff Distance (HD), mikan also computes related metrics like the Average Symmetric Surface Distance (ASSD) and the 95th percentile Hausdorff Distance (HD95). There's no need to calculate them separately!

In [14]:
t = time.time()
mikan_hd = e.labels(1).metrics("assd")
print(f"Mikan cost {time.time() - t:.4f} s, ASSD = {mikan_hd:.4f} mm")

Mikan cost 0.0000 s, ASSD = 0.4627 mm

In contrast, medpy requires starting from scratch to calculate the ASSD, which can be very slow:

In [15]:
from medpy.metric import assd
t = time.time()
medpy_hd = assd(pred_arr == 1, gt_arr == 1, voxelspacing=gt.GetSpacing()[::-1]) # z, y, x
print(f"medpy cost {time.time() - t:.2f} s, ASSD = {medpy_hd:.4f} mm")

medpy cost 125.13 s, ASSD = 0.4627 mm

Here's another example with the Dice coefficient:

In [16]:
from medpy.metric import dc

# mikan: DSC
t = time.time()
mikan_dscs = e.labels([1,2,3,4,5]).metrics("dsc")
mikan_costs = time.time() - t + 1e-10 # sometimes zero

# medpy: DSC
t = time.time()
dscs = []
for i in (1,2,3,4,5):
    dscs.append(dc(pred_arr == i, gt_arr == i))
medpy_costs = time.time() - t

print(f"mikan DSC: {medpy_costs / mikan_costs :.2f}x faster")

mikan DSC: 2590.26x faster

mikan won't recalculate data that has already been processed. This is thanks to mikan's super caching feature!

### Aliases

Mikan also supports aliases. For example, you can refer to "Dice" as "dsc": 

In [17]:
e.labels(1).metrics("dsc") # dsc is also Dice！

0.9870104666571462

We maintain an alias dictionary, allowing you to use any name on the left side, and they will be interpreted as the corresponding name on the right for calculations.

In [18]:
from mikan.alias import ALIAS_DICT
print(ALIAS_DICT)

{
    'accuracy': 'accuracy',
    'acc': 'accuracy',
    'Acc': 'accuracy',
    'ACC': 'accuracy',
    'Accuracy': 'accuracy',
    'RI': 'accuracy',
    'RandIndex': 'accuracy',
    'adjust_rand_score': 'adjust_rand_score',
    'auc': 'auc',
    'AUC': 'auc',
    'AUC_trapezoid': 'auc',
    'balanced_accuracy': 'balanced_accuracy',
    'bacc': 'balanced_accuracy',
    'BACC': 'balanced_accuracy',
    'BalancedAccuracy': 'balanced_accuracy',
    'dice': 'dice',
    'Dice': 'dice',
    'dsc': 'dice',
    'DSC': 'dice',
    'f_score': 'f_score',
    'jaccard_score': 'jaccard_score',
    'iou': 'jaccard_score',
    'IoU': 'jaccard_score',
    'jaccard': 'jaccard_score',
    'Jaccard': 'jaccard_score',
    'IntersectionOverUnion': 'jaccard_score',
    'kappa': 'kappa',
    'KAP': 'kappa',
    'Kappa': 'kappa',
    'CohensKappa': 'kappa',
    'mcc': 'mcc',
    'MCC': 'mcc',
    'MatthewsCorrelationCoefficient': 'mcc',
    'nmcc': 'nmcc',
    'NMCC': 'nmcc',
    'nMCC': 'nmcc',
    'MCC_normalized': 'nmcc',
    'amcc': 'amcc',
    'AMCC': 'amcc',
    'aMCC': 'amcc',
    'MCC_absolute': 'amcc',
    'precision': 'precision',
    'prec': 'precision',
    'PREC': 'precision',
    'Precision': 'precision',
    'senstivity': 'senstivity',
    'sens': 'senstivity',
    'SENS': 'senstivity',
    'Sensitivity': 'senstivity',
    'Recall': 'senstivity',
    'recall': 'senstivity',
    'TPR': 'senstivity',
    'tpr': 'senstivity',
    'true_positive_rate': 'senstivity',
    'specificity': 'specificity',
    'Specificity': 'specificity',
    'spec': 'specificity',
    'Spec': 'specificity',
    'SPEC': 'specificity',
    'TNR': 'specificity',
    'tnr': 'specificity',
    'true_negative_rate': 'specificity',
    'tn': 'tn',
    'TN': 'tn',
    'TrueNegative': 'tn',
    'tp': 'tp',
    'TP': 'tp',
    'TruePositive': 'tp',
    'fn': 'fn',
    'FN': 'fn',
    'FalseNegative': 'fn',
    'fp': 'fp',
    'FP': 'fp',
    'FalsePositive': 'fp',
    'fnr': 'fnr',
    'FNR': 'fnr',
    'fpr': 'fpr',
    'FPR': 'fpr',
    'volume_similarity': 'volume_similarity',
    'VolumetricSimilarity': 'volume_similarity',
    'vs': 'volume_similarity',
    'VS': 'volume_similarity',
    'hausdorff_distance_95': 'hausdorff_distance_95',
    'HD95': 'hausdorff_distance_95',
    'hd95': 'hausdorff_distance_95',
    'hausdorff_distance': 'hausdorff_distance',
    'hd': 'hausdorff_distance',
    'HD': 'hausdorff_distance',
    'assd': 'assd',
    'ASSD': 'assd',
    'masd': 'masd',
    'MASD': 'masd'
}

### Result Consistency

The results are completely consistent with medpy. You can use mikan with confidence!

### Summary

- Mikan can calculate almost all segmentation metrics.
- Mikan is exceptionally fast.
- Mikan is very easy to use.

